# GGR376 LAB 5.3: Creating a time series animation using Python (2 marks)

In this final section of the lab we will be using the [express](https://plotly.com/python/plotly-express/) module from the **plotly** library to create simple animated visualizations. We will work with point and linear representations of historical tornado data in Ontario (as used in Lab 3).

In [ ]:
# Import libraries
import geopandas as gpd
import pandas as pd
import numpy as np
import plotly.express as px
import shapely.geometry

In [ ]:
# Manually point to projections library to avoid error when reading shapefile
import os
os.environ['PROJ_LIB'] = '/opt/conda/share/proj'

In [ ]:
# Read in the shapefile of tornado points and view the top 5 rows
tornadopnts = gpd.read_file('data/tornadopoints.shp')
tornadopnts.head()

### Explore and prepare the data for animation

In [ ]:
# Check the points plot as expected using basic geopandas plot method
tornadopnts.plot()

In [ ]:
# Create new field to store the year each tornado was recorded
tornadopnts['year'] = tornadopnts['YYYYMMDDHH'].str[:4] # str[] function reads the specified number of characters from the left of a text field
tornadopnts.head()

### Animate tornado point locations
Our point data are ready for animation! Below is a simple block of code that creates an animated map of tornado points by year. Click on the scatter_geo function and hold SHIFT + TAB keys to explore the possible parameters. You may also refer to the documentation for applied examples of [scatter_geo](https://plotly.com/python/scatter-plots-on-maps/).

In [ ]:
# Create animated scatter plot of tornado points by year
fig = px.scatter_geo(tornadopnts,
                    lat=tornadopnts.geometry.y,
                    lon=tornadopnts.geometry.x,
                    animation_frame='year')

fig.update_layout(
        title = 'Tornado start points in Southern Ontario<br>(1980 to 2009)',
        geo_scope = 'north america')


fig.show()


#Zoom in to Ontario on the map to see animation more clearly

### Your turn (1 mark):
Update the code to include at least two additional parameters. For example, you may set the size, color, or add a pop-up on hover. Add comments explaining what your new parameters do.

In [ ]:
### EXAMPLE UPDATES
fig = px.scatter_geo(tornadopnts,
                    lat=tornadopnts.geometry.y,
                    lon=tornadopnts.geometry.x,
                    hover_name='NEAR_CMMTY',
                    size='LENGTH_M',
                    color='year',
                    animation_frame='year')

fig.update_layout(
        title = 'Tornado start points in Southern Ontario<br>(1980 to 2009)',
        geo_scope = 'north america')


fig.show()

### Animate tornado tracks
Next we will work with the linear tornado tracks. Animating linear features is a little trickier than points but you will start to gain an understanding of how it is possible to animate one line at a time.

In [ ]:
# Read in the Tornado tracks shapefile
tornadotrks = gpd.read_file('data/tornadotracks.shp')
tornadotrks.head()

In [ ]:
# Check the plot using pandas plot method
tornadotrks.plot()

Line maps can be animated with plotly express using arrays of XY values. Remember from part 2 of the lab that the numpy library is useful for working with arrays.

It is possible to extract data from a GeoDataFrame and store data as arrays. The code below initializes empty arrays and then loops through each row of the GeoDataFrame, storing values from select fields.

**Review the following code block and add see if you can add comments to each line, explaining what each section does.** This will not be graded but you will find it helpful for improving your understanding of Python. Use the python handbook for reference.

In [ ]:
lats = []
lons = []
names = []
dates = []

for feature, name, date in zip(tornadotrks.geometry, tornadotrks.NEAR_CMMTY, tornadotrks.YYYYMMDDHH):
    if isinstance(feature, shapely.geometry.linestring.LineString):
        linestrings = [feature]
    else:
        continue
    for linestring in linestrings:
        x, y = linestring.xy
        lats = np.append(lats, y)
        lons = np.append(lons, x)
        names = np.append(names, [name]*len(y))
        dates = np.append(dates, [date]*len(y))
        lats = np.append(lats, None)
        lons = np.append(lons, None)
        names = np.append(names, None)
        dates = np.append(dates, None)


In [ ]:
### EXAMPLE COMMENTS

# Create empty arrays for lats, longs, dates, and area names
lats = []
lons = []
names = []
dates = []

# Iterate through geometry features, area names, and dates of tornadoes in geodataframe
for feature, name, date in zip(tornadotrks.geometry, tornadotrks.NEAR_CMMTY, tornadotrks.YYYYMMDDHH):
    if isinstance(feature, shapely.geometry.linestring.LineString):
        linestrings = [feature] # If the feature is a linestring, create a list of the feature
    else:
        continue # Or skip to next iteration
    for linestring in linestrings: # For each item in the list...
        x, y = linestring.xy # Extract XY coords
        lats = np.append(lats, y) # Add coord values to array
        lons = np.append(lons, x)
        names = np.append(names, [name]*len(y)) # Add corresponding name for lat-lon pairs
        dates = np.append(dates, [date]*len(y))
        lats = np.append(lats, None) # Append 'None' to separate linestrings
        lons = np.append(lons, None)
        names = np.append(names, None)
        dates = np.append(dates, None)

### Your turn (1 mark):
Use the plotly express .line_geo() function to animate the tornado tracks. 

*Hint: refer to the scatter_geo code for guidance and make sure you set the lat, lon, and animation_frame parameters to your new arrays. Arrays do not have to be contained in single quotation marks like fieldnames.*

Notice that a single linear feature appears for each timestamp, even if mutliple tornado tracks are recorded at the same day and time. This happens because the code merges arrays of XY values in the plot. If an array were created for years and the animation frame were set to years, all tracks recorded within the same year would be merged. You can test this out by updateing the date field from *YYYYMMDDHH* to *TornadoDat*.

Once you have completed the lab, export your notebook by selection File > Print Preview > Save as PDF 

Upload your PDF to Part 3 of the lab.